In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
%load_ext autoreload
%autoreload 2



In [5]:
import torch
from diffusers import StableDiffusionPipeline
from stablediffusion_dixit.image_generation.local_generation.progress_capturer import ProgressCapturer




if not torch.cuda.is_available():
    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", safety_checker=None)
    if torch.backends.mps.is_available():
        pipe = pipe.to("mps")
else:
    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, safety_checker=None).to("cuda:0")


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

You have passed `None` for safety_checker to disable its functionality in <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'>. Note that this might lead to problems when using <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> and is not recommended.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [1]:
pc = ProgressCapturer(pipe)
i = pipe("an astronaut riding a horse, abstract digital art", num_inference_steps=20, width=384, height=512, callback=pc).images
pci = pc.get_images()
i[0]

NameError: name 'ProgressCapturer' is not defined

In [ ]:
with torch.no_grad():
    latents = torch.stack(pc.latents).reshape(11, 4, 64, 48)[9:11]

    latents = 1 / 0.18215 * latents
    images = pipe.vae.decode(latents).sample

    images = (images / 2 + 0.5).clamp(0, 1)

    images = images.cpu().permute(0, 2, 3, 1).float().numpy()
    p = pipe.numpy_to_pil(images)


In [ ]:
pci[10]

In [11]:
pci[2].save('generation.gif', format='GIF',
               append_images=pci[3:], save_all=True, duration=300, loop=0)

In [ ]:
from RealESRGAN import RealESRGAN

In [ ]:
model = RealESRGAN("mps", scale=2)
model.load_weights("weights/RealESRGAN_x2.pth", download=True)

In [ ]:
model.predict(i[0].convert("RGB"))

In [1]:
from stablediffusion_dixit.image_generation.local_generation.local_image_generator import LocalImageGenerator


In [2]:
l = LocalImageGenerator()

In [10]:
l.request_generation("an astronaut riding a horse", lambda idx, path1, path2: print(f"{idx}: {path1}, {path2}"))


2

In [9]:
l.get_image_and_anim(0)